# Getting Started

We start by importing the library and configuring matplotlib to display the figures in a nice way.

In [ ]:
import sys; sys.path.append('..')
import random
import pyzx as zx

%config InlineBackend.figure_format = 'svg'

Then we can create a new random Clifford circuit and visualise it.

In [ ]:
qubit_amount = 4
depth = 15
random.seed(1337)
circ = zx.generate.cliffords(qubit_amount, depth)
zx.draw(circ,labels=True,h_edge_draw='box')

The green and red nodes represent Z- and X-phase gates respectively, the yellow boxes are Hadamard gates, and the vertical lines going between two different colored nodes are CNOT gates.

Internally this circuit is represented as a graph:

In [ ]:
print(circ)
print("All edges: ", list(circ.edges()))
print("\nThe neighbours of a particular vertex: ", list(circ.neighbours(14)))

Using this graph representation we can use the rules of the ZX-calculus to simplify it:

In [ ]:
g = circ.copy()
zx.clifford_simp(g)

In [ ]:
g.normalise() # Reposition nodes horizontally to look nicer
zx.draw(g)

The blue lines represent edges that have a Hadamard gate on them.

Let's try to see in a bit more detail what goes into rewriting this circuit.

In [ ]:
from ipywidgets import widgets
from IPython.display import display, Markdown

from pyzx.simplify import clifford_iter

graph = circ.copy()
graphs = [zx.draw(graph)]
names = ["start"]
for gs, n in clifford_iter(graph):
    graphs.append(zx.draw(gs))
    names.append(n)
zx.drawing.pack_circuit_nf(gs,'grg')
graphs.append(zx.draw(gs))
names.append("reposition")

def plotter(rewrite):
    display(Markdown("Rewrite step: " + names[rewrite]))
    display(graphs[rewrite])

w = widgets.interactive(plotter, rewrite=(0,len(graphs)-1))
slider = w.children[0]
slider.layout.width = "{!s}px".format(min(800,50*len(graphs)))
output = w.children[-1]
output.layout.height = "{!s}px".format(200+qubit_amount*20)
slider.value = 0
w

As you can see, the first few steps are **spider fusion**, where adjacent red nodes and adjacent green nodes are fused together (and their phases added). The next step is to convert all red nodes to green nodes by changing regular edges to hadamard edges.

Now come the steps that do most of the work. These steps are a round of **pivots** followed by **local complementations** and then a round of pivots again. The local complementation rules looks for green nodes with a phase of $\frac\pi2$ or $\frac{3\pi}2$. By doing a local complementation on this node we can remove the node from the graph. This process is done until no more suitable candidates are found. A pivot consists of a series of three local complementations that is done on two neighbouring vertices that have a zero or $\pi$ phase. A pivot move allows us to remove these two nodes.

Finally we do a round of **identity** simplification which removes phaseless nodes that are connected to precisely two neighbours.

**Note**: Depending on the circuit that was generated, any of these steps might not have been done.

Even though this graph is a lot compacter than the one we started out with, it no longer looks like a circuit. To fix this we need to be clever.

In [ ]:
circ2 = g.copy()
circ2.normalise()
zx.extract.clifford_extract(circ2,1,2)
zx.draw(circ2)

To verify that this circuit is still equal to the original circuit, we can transform them into numpy tensors and compare these tensors for equality:

In [ ]:
t1 = circ.to_tensor()
t2 = circ2.to_tensor()

zx.compare_tensors(t1,t2) # This checks whether t1 and t2 are equal up to some number: t1 == z*t2 for some complex number z

We can represent this circuit-like ZX-graph as an actual circuit using PyZX:

In [ ]:
c = zx.Circuit.from_graph(circ2)
print(c.gates)

And we can represent this in one of several quantum circuit description languages, such as that of QUIPPER:

In [ ]:
print(c.to_quipper())

Now lets try the same thing with a more complicated Clifford+T circuit.

In [ ]:
qubit_amount = 6
depth = 70
random.seed(1338)
circ = zx.generate.cliffordT(qubit_amount, depth,p_t=0.2)
zx.draw(circ,figsize=(17,2),h_edge_draw='box')

In [ ]:
g = circ.copy()
zx.clifford_simp(g, quiet=True)
g.normalise()
zx.draw(g)

We use a different extraction procedure for this circuit. Instead of producing a ZX-graph as output we get a description as a circuit:

In [ ]:
g2 = g.copy()
c = zx.extract.streaming_extract(g2)
print(c.stats())

We can convert this back into a PyZX-graph:

In [ ]:
zx.draw(c.to_graph(), figsize=(20,3))

And verify that it is still equal to the original graph:

In [ ]:
zx.compare_tensors(c.to_tensor(), circ.to_tensor())

Now lets represent this circuit in the QASM circuit description language:

In [ ]:
print(c.to_basic_gates().to_qasm())